In [178]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [179]:
# columns = ["_time", "_measurement", "packet_interarrival_time", "packet_jitter_weighted", "packet_owd"]
num_cols = ["packet_interarrival_time", "packet_jitter_weighted", "packet_owd"]
str_cols = ["_time", "_measurement"]
columns = num_cols + str_cols

In [180]:
def get_aggregated_df(df, point_lim):
    group_size = df.shape[0] // point_lim 
    df['group'] = pd.Series(range(df.shape[0])) // group_size

    aggregated_mean_df = df.groupby('group')[num_cols].mean().rename(columns={col: f"{col}_mean" for col in num_cols})
    aggregated_std_df = df.groupby('group')[num_cols].std().rename(columns={col: f"{col}_std" for col in num_cols})
    aggregated_str_df = df.groupby('group')[str_cols].first()

    aggregated_df = pd.concat([aggregated_mean_df, aggregated_std_df, aggregated_str_df], axis=1).reset_index(drop=True)

    aggregated_df['timestamp'] = pd.to_datetime(aggregated_df['_time'], format="ISO8601")
    aggregated_df.set_index('timestamp', inplace=True)

    min_ts = aggregated_df.index.min()

    aggregated_df.index = (aggregated_df.index - min_ts).total_seconds()

    return aggregated_df

In [181]:
POINT_LIM = 250
TRIAL_NAME = "cbr_30kpps"

In [182]:
print(df1)

         packet_interarrival_time  packet_jitter_weighted  packet_owd  \
0                    1.883507e-05                0.000000         NaN   
1                    1.001358e-05                0.000000         NaN   
2                    1.001358e-05                0.000000         NaN   
3                    1.001358e-05                0.000000         NaN   
4                    1.001358e-05                0.000000         NaN   
...                           ...                     ...         ...   
1799140              1.907349e-06                0.001808    2.216365   
1799141              2.145767e-06                0.006644    2.295558   
1799142              9.536743e-07                0.006271    2.296231   
1799143              1.907349e-06                0.005881    2.296263   
1799144                       NaN                0.000000         NaN   

                                  _time         _measurement  group  
0        2024-05-09T09:47:47.808404922Z  cbr_30kpps_a

In [183]:
df1, df2 = \
    pd.read_csv(f"/home/sebastfu/run_1/{TRIAL_NAME}.csv", skiprows=3)[columns],\
    pd.read_csv(f"/home/sebastfu/run_2/{TRIAL_NAME}.csv", skiprows=3)[columns]
    # pd.read_csv(f"/home/sebastfu/run_3/{TRIAL_NAME}.csv", skiprows=3)[columns],\
    # pd.read_csv(f"/home/sebastfu/run_4/{TRIAL_NAME}.csv", skiprows=3)[columns],\
    # pd.read_csv(f"/home/sebastfu/run_5/{TRIAL_NAME}.csv", skiprows=3)[columns]

# df1["_time"] = pd.to_datetime(df1['_time'])
# df2["_time"] = pd.to_datetime(df2['_time'])
# df3["_time"] = pd.to_datetime(df3['_time'])
# df4["_time"] = pd.to_datetime(df4['_time'])
# df5["_time"] = pd.to_datetime(df5['_time'])

df1["_measurement"] = df1["_measurement"] + "_1"
df2["_measurement"] = df2["_measurement"] + "_2"
# df3["_measurement"] = df3["_measurement"] + "_3"
# df4["_measurement"] = df4["_measurement"] + "_4"
# df5["_measurement"] = df5["_measurement"] + "_5"

agg_df1, agg_df2 = \
    get_aggregated_df(df1, POINT_LIM),\
    get_aggregated_df(df2, POINT_LIM)
    # get_aggregated_df(df3, POINT_LIM)#,\
    # get_aggregated_df(df4, POINT_LIM),\
    # get_aggregated_df(df5, POINT_LIM)


In [184]:
def calculate_mean_across_dfs(df1, df2, df3, column):
    # Choose one of the input DataFrames to get the index
    means = []
    for i in range(251):
        # columnSum = df1[column].iloc[i] + df2[column].iloc[i] + df3[column].iloc[i] + df4[column].iloc[i] + df5[column].iloc[i]
        columnSum = df1[column].iloc[i] + df2[column].iloc[i] + df3[column].iloc[i]
        avg = columnSum/3
        means.append(avg)
    
    mean_df = pd.DataFrame(index=df1.index, data={column: means})
    mean_df["_measurement"] = "mean"
    return mean_df


In [185]:
# mean_df = calculate_mean_across_dfs(agg_df1, agg_df2, agg_df3, "packet_owd_mean")

In [186]:
# new_agg_df = pd.concat([agg_df1, agg_df2, agg_df3, mean_df], axis=0)
new_agg_df = pd.concat([agg_df1, agg_df2], axis=0)


In [187]:
fig = px.line(x=new_agg_df.index, y=new_agg_df['packet_owd_mean'], color=new_agg_df['_measurement'], template='simple_white')

fig.update_xaxes(
    title="Timestamp (s)",
    showline=True,
    linecolor="black",
    linewidth=1,
    mirror=True,
    gridcolor="LightGray",
    showgrid=True
)
fig.update_yaxes(
    title="One-way delay (s)",
    showline=True,
    linecolor="black",
    linewidth=1,
    mirror=True,
    gridcolor="LightGray",
    showgrid=True
)

fig.update_layout(
    legend_title_text="Run index"
)

# 'newLegendNames = {
#     f'{TRIAL_NAME}_analysis_1': '1',
#     f'{TRIAL_NAME}_analysis_2': '2',
#     f'{TRIAL_NAME}_analysis_3': '3',
#     f'{TRIAL_NAME}_analysis_4': '4',
#     f'{TRIAL_NAME}_analysis_5': '5',
#     'mean': 6
# }

# fig.for_each_trace(lambda t: t.update(name=newLegendNames[t.name]))
# '
fig.show()

In [188]:
px.line(x=new_agg_df.index, y=new_agg_df['packet_jitter_weighted_mean'], color=new_agg_df['_measurement'])

In [189]:
agg_columns = ['_time', '_measurement', 'packet_loss', 'throughput']
analysis_aggregate_df_1, analysis_aggregate_df_2, analysis_aggregate_df_3 = \
    pd.read_csv("cbr_1000pps/cbr_low_bw_med_dur_low_pl_1_analysis_aggregate.csv", skiprows=3)[agg_columns],\
    pd.read_csv("cbr_1000pps/cbr_low_bw_med_dur_low_pl_2_analysis_aggregate.csv", skiprows=3)[agg_columns],\
    pd.read_csv("cbr_1000pps/cbr_low_bw_med_dur_low_pl_3_analysis_aggregate.csv", skiprows=3)[agg_columns]

analysis_aggregate_df_1['timestamp'] = pd.to_datetime(analysis_aggregate_df_1['_time'], format="ISO8601")
analysis_aggregate_df_1.set_index('timestamp', inplace=True)
min_ts = analysis_aggregate_df_1.index.min()
analysis_aggregate_df_1.index = (analysis_aggregate_df_1.index - min_ts).total_seconds()

analysis_aggregate_df_2['timestamp'] = pd.to_datetime(analysis_aggregate_df_2['_time'], format="ISO8601")
analysis_aggregate_df_2.set_index('timestamp', inplace=True)
min_ts = analysis_aggregate_df_2.index.min()
analysis_aggregate_df_2.index = (analysis_aggregate_df_2.index - min_ts).total_seconds()

analysis_aggregate_df_3['timestamp'] = pd.to_datetime(analysis_aggregate_df_3['_time'], format="ISO8601")
analysis_aggregate_df_3.set_index('timestamp', inplace=True)
min_ts = analysis_aggregate_df_3.index.min()
analysis_aggregate_df_3.index = (analysis_aggregate_df_3.index - min_ts).total_seconds()

new_agg_df_aggregate = pd.concat([analysis_aggregate_df_1, analysis_aggregate_df_2, analysis_aggregate_df_3], axis=0)

FileNotFoundError: [Errno 2] No such file or directory: 'cbr_1000pps/cbr_low_bw_med_dur_low_pl_1_analysis_aggregate.csv'

In [ ]:
fig = px.line(x=new_agg_df_aggregate.index, y=new_agg_df_aggregate['packet_loss'], color=new_agg_df_aggregate['_measurement'], template='simple_white')

fig.update_xaxes(
    title="Timestamp (s)",
    showline=True,
    linecolor="black",
    linewidth=1,
    mirror=True,
    gridcolor="LightGray",
    showgrid=True
)
fig.update_yaxes(
    title="One-way delay (s)",
    showline=True,
    linecolor="black",
    linewidth=1,
    mirror=True,
    gridcolor="LightGray",
    showgrid=True
)

fig.update_layout(
    legend_title_text="Run index"
)

newLegendNames = {
    'cbr_low_bw_med_dur_low_pl_1_analysis_aggregate': '1',
    'cbr_low_bw_med_dur_low_pl_2_analysis_aggregate': '2',
    'cbr_low_bw_med_dur_low_pl_3_analysis_aggregate': '3'
}

fig.for_each_trace(lambda t: t.update(name=newLegendNames[t.name]))

fig.show()

In [ ]:
px.line(x=new_agg_df_aggregate.index, y=new_agg_df_aggregate['throughput'], color=new_agg_df_aggregate['_measurement'])